# Prediction mit Hilfe des Models Age Detection
## Schritt 1: Einlesen der CSV Dateien mit extrahierten Daten der Testpersonen
Anpassen der Variable Alter
--> Nachdem die Schneidung in 10er Jahre nicht aussagekräftig genug war, werden in einem zweiten Versuch 20er Jahre zusammengefasst:  
0: Best Ager (40-59 Jahre) entspricht vorher: fourties + fifties  
1: Mid Ager (20-39 Jahre) entspricht vorher: twenties + thirties  
2: Senior (59+ Jahre) entspricht vorher: sixties + seventies  
3: Youngster (bis 19 Jahre) enspricht vorher: teens + twenties  
## Schritt 2: Liste mit benötigten Spaltennamen extrahieren
## Schritt 3: Formatierung der Werte in float
## Schritt 4: Entfernen der nicht mehr benötigten Spalten für die Prediction

In [5]:
import pandas as pd
import numpy as np

#df_test = pd.read_csv ('/home/ec2-user/SageMaker/voice_gender_age/validation_test/test_final_FeaturesExtracted.csv', delimiter = ';')
df_test = pd.read_csv ('/home/ec2-user/SageMaker/voice_gender_age/validation_test/FeaturesExtracted_TestdatenBDC_Gender_Age.csv', delimiter = ';')

list_cn = df_test.columns.values
list_cn = np.delete(list_cn,0)
list_cn = np.delete(list_cn,0)
list_cn = np.delete(list_cn,0)
list_cn = np.delete(list_cn,0)
list_cn = np.delete(list_cn,0)
list_cn = np.delete(list_cn,0)
list_cn

nrows = len(df_test)

for i in range (0,nrows):
    for j in list_cn:
        list_c = df_test[j].iloc[i]
        list_c = list_c.replace('[','')
        list_c = list_c.replace(']','')
        list1 = np.fromstring (list_c, sep=' ',dtype=float)
        df_test[j].iat[i] = list1
        df_test[j].iat[i] = df_test[j].iloc[i].mean()
    if (i % 10000 == 0):
        print ("Reihe "+ str(i) + " fertig")
print ("Konvertierung abgeschlossen.")

print (df_test.shape)

df_pred = df_test.copy()
del df_pred['Unnamed: 0']
del df_pred['index']
del df_pred['path']
del df_pred['age']
del df_pred['file_wave']
del df_pred['gender']

#list (df_pred.columns)
#df_pred.shape

print ("Fertig !!!")

Reihe 0 fertig
Konvertierung abgeschlossen.
(11, 40)
Fertig !!!


## Prediction der Testwerte
Die Ausgabe der Prediction gibt die vorhergesagte Klasse wieder (prediction_class)
0 = Male
1 = Female
Die Variable prediction_proba gibt die errechnete Wahrscheinlichkeit aus, dass es sich um die vorhergesagte Klasse handelt

Schritt 1: Trainiertes Modell laden

In [2]:
import tensorflow as tf
from tensorflow import keras
# model_age = keras.models.load_model ('/home/ec2-user/SageMaker/voice_gender_age/Age_without_Gender_opti/model.h5', custom_objects = None, compile = True)
# Neues Modell mit breiter Spanne
model_age = keras.models.load_model ('/home/ec2-user/SageMaker/voice_gender_age/Age_without_Gender_opti/model_breiteSpanne.h5', custom_objects = None, compile = True)

# model_age.summary()

df_final = df_test.copy()
df_final['pred_class']=0
df_final['pred_proba']=0

for i in range (7):
    prediction = model_age.predict(df_pred.iloc[[i]])
    prediction_class = model_age.predict_classes(df_pred.iloc[[i]])
    prediction_proba = model_age.predict_proba(df_pred.iloc[[i]])
    print(prediction)
    print (prediction_class)
    df_final.at[i,'pred_class'] = prediction_class
    #df_final.at[i,'pred_proba'] = prediction_proba
    #print (prediction_proba)
    print ("----------")
    
anzahl_samples = df_final.shape[0]
korrekt = 0
falsch = 0

## Mit kleiner Spanne
#for i in range (anzahl_samples):
    #print (str(df_final_age_w_gender.pred_class[i]) + ' - ' + str(df_final_age_w_gender.age[i]))
#    if ((df_final.pred_class[i]==4) & (df_final.age[i] == 'teens')):
#        korrekt +=1
#    elif ((df_final.pred_class[i]==6) & (df_final.age[i] == 'twenties')):
#        korrekt +=1
#    elif ((df_final.pred_class[i]==5) & (df_final.age[i] == 'thirties')):
#        korrekt +=1
#    elif ((df_final.pred_class[i]==1) & (df_final.age[i] == 'fourties')):
#        korrekt +=1
#    elif ((df_final.pred_class[i]==0) & (df_final.age[i] == 'fifties')):
#        korrekt +=1
#    elif ((df_final.pred_class[i]==3) & (df_final.age[i] == 'sixties')):
#        korrekt +=1
#    elif ((df_final.pred_class[i]==2) & (df_final.age[i] == 'seventies')):
#        korrekt +=1
#    else:
#        falsch +=1

## Mit großer Spanne
for i in range (anzahl_samples):
    if ((df_final.pred_class[i]==3) & (df_final.age[i] == 'teens')):
        korrekt +=1
    elif ((df_final.pred_class[i]==1) & (df_final.age[i] == 'twenties' or df_final.age[i] == 'thirties')):
        korrekt +=1
    elif ((df_final.pred_class[i]==0) & (df_final.age[i] == 'fourties' or df_final.age[i] == 'fifties')):
        korrekt +=1
    elif ((df_final.pred_class[i]==2) & (df_final.age[i] == 'sixties' or df_final.age[i] == 'seventies')):
        korrekt +=1
    else:
        falsch +=1

print ('Korrekt: ' + str(korrekt) +'/'+ str(anzahl_samples))
print ('Falsch: ' + str(falsch) +'/'+ str(anzahl_samples))


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
[[0.3838395  0.60889417 0.00509487 0.00217148]]
[1]
----------
[[3.6316228e-01 6.3474733e-01 3.5222192e-04 1.7381496e-03]]
[1]
----------
[[0.32991913 0.65469897 0.00320573 0.01217616]]
[1]
----------
[[9.2040652e-01 7.9348892e-02 3.6174877e-10 2.4452771e-04]]
[0]
----------
[[8.7349135e-01 1.2283887e-01 2.2472213e-12 3.6698100e-03]]
[0]
----------
[[7.6585769e-04 9.4835901e-01 2.5584912e-10 5.0875153e-02]]
[1]
----------
[[5.1908067e-04 9.9947995e-01 5.6427610e-12 1.0089118e-06]]
[1]
----------
Korrekt: 3/11
Falsch: 8/11


## Details
Die Codierung des Alters im Model war wie folgt:
## Enge Spanne
teens: 4  
twenties: 6  
thirties: 5  
fourties: 1  
fifties: 0  
sixties: 3  
seventies: 2  

Wie viel wurde korrekt erkannt?  
6/7

## Breite Spanne
0: Best Ager (40-59 Jahre) entspricht vorher: fourties + fifties  
1: Mid Ager (20-39 Jahre) entspricht vorher: twenties + thirties  
2: Senior (59+ Jahre) entspricht vorher: sixties + seventies  
3: Youngster (bis 19 Jahre) enspricht vorher: teens + twenties  

Korrekt erkannt:  
6/7

# Prediction mit Age und Gender Erkennung

Schritt 1: Einlesen des Modells, welches für Alter und Geschlecht erstellt wurde

In [ ]:
## Enge Spanne
# model_age_a_gender = keras.models.load_model ('/home/ec2-user/SageMaker/voice_gender_age/Age_with_Gender_opti/model.h5', custom_objects = None, compile = True)

## Breite Spanne
model_age_a_gender = keras.models.load_model ('/home/ec2-user/SageMaker/voice_gender_age/Age_with_Gender_opti/model_großeSpanne.h5', custom_objects = None, compile = True)
print ("Modell eingelesen. Übersicht:")
model_age_a_gender.summary()

#df_test_age_a_gender = pd.read_csv ('/home/ec2-user/SageMaker/voice_gender_age/validation_test/test_final_FeaturesExtracted.csv', delimiter = ';')
df_test_age_a_gender = pd.read_csv ('/home/ec2-user/SageMaker/voice_gender_age/validation_test/FeaturesExtracted_TestdatenBDC_Gender_Age.csv', delimiter = ';')

print ("Datei eingelesen. Vorschau:")
df_test_age_a_gender.head()
print ("--------------------")
list_cn = df_test_age_a_gender.columns.values
list_cn = np.delete(list_cn,0)
list_cn = np.delete(list_cn,0)
list_cn = np.delete(list_cn,0)
list_cn = np.delete(list_cn,0)
list_cn = np.delete(list_cn,0)
list_cn = np.delete(list_cn,0)
list_cn

nrows = len(df_test_age_a_gender)

for i in range (0,nrows):
    for j in list_cn:
        list_c = df_test_age_a_gender[j].iloc[i]
        list_c = list_c.replace('[','')
        list_c = list_c.replace(']','')
        list1 = np.fromstring (list_c, sep=' ',dtype=float)
        df_test_age_a_gender[j].iat[i] = list1
        df_test_age_a_gender[j].iat[i] = df_test_age_a_gender[j].iloc[i].mean()
    if (i % 10000 == 0):
        print ("Reihe "+ str(i) + " fertig")
print ("Konvertierung abgeschlossen.")
print ("--------------------")
df_pred_age_w_gender = df_test_age_a_gender.copy()
del df_pred_age_w_gender['Unnamed: 0']
del df_pred_age_w_gender['index']
del df_pred_age_w_gender['path']
del df_pred_age_w_gender['age']
del df_pred_age_w_gender['file_wave']

import re
df_pred_age_w_gender['gender']= df_pred_age_w_gender['gender'].apply(lambda x: ' '.join([re.sub('female','1', x)]))
df_pred_age_w_gender['gender']= df_pred_age_w_gender['gender'].apply(lambda x: ' '.join([re.sub('male','0', x)]))
df_pred_age_w_gender.head()

df_final_age_w_gender = df_test_age_a_gender.copy()
df_final_age_w_gender['pred_class']=0
df_final_age_w_gender['pred_proba']=0

anz_datensaetze = len(df_final_age_w_gender)
for i in range (anz_datensaetze):
    prediction = model_age_a_gender.predict(df_pred_age_w_gender.iloc[[i]])
    prediction_class = model_age_a_gender.predict_classes(df_pred_age_w_gender.iloc[[i]])
    prediction_proba = model_age_a_gender.predict_proba(df_pred_age_w_gender.iloc[[i]])
    #print(prediction)
    #print (prediction_class)
    df_final_age_w_gender.at[i,'pred_class'] = prediction_class
    #df_final.at[i,'pred_proba'] = prediction_proba
    #print (prediction_proba)
   # print ("----------")
    print('--------------------------------------')
    print('Datensatz Nummer (' + str(i) + '):')
    print('--------------------------------------')
    print('Klasse 0 (fourties + fifties): ' + str(prediction[0,0]))
    print('Klasse 1 (twenties + thirties): ' + str(prediction[0,1]))
    print('Klasse 2 (sixties + seventies): ' + str(prediction[0,2]))
    print('Klasse 3 (teens): ' + str(prediction[0,3]))
    print('-------------------------------------------')
    print('Prediction als Klasse: ' + str(prediction_class[0]))

print ("--------------------")
print (df_final_age_w_gender[['gender','age','pred_class']])
print ("--------------------")

anzahl_samples = df_final_age_w_gender.shape[0]
korrekt = 0
falsch = 0

#for i in range (anzahl_samples):
#    if ((df_final_age_w_gender.pred_class[i]==4) & (df_final_age_w_gender.age[i] == 'teens')):
#        korrekt +=1
#    elif ((df_final_age_w_gender.pred_class[i]==6) & (df_final_age_w_gender.age[i] == 'twenties')):
#        korrekt +=1
#    elif ((df_final_age_w_gender.pred_class[i]==5) & (df_final_age_w_gender.age[i] == 'thirties')):
#        korrekt +=1
#    elif ((df_final_age_w_gender.pred_class[i]==1) & (df_final_age_w_gender.age[i] == 'fourties')):
#        korrekt +=1
#    elif ((df_final_age_w_gender.pred_class[i]==0) & (df_final_age_w_gender.age[i] == 'fifties')):
#        korrekt +=1
#    elif ((df_final_age_w_gender.pred_class[i]==3) & (df_final_age_w_gender.age[i] == 'sixties')):
#        korrekt +=1
#    elif ((df_final_age_w_gender.pred_class[i]==2) & (df_final_age_w_gender.age[i] == 'seventies')):
#        korrekt +=1
#    else:
#        falsch +=1

        ## Mit großer Spanne
for i in range (anzahl_samples):
    if ((df_final_age_w_gender.pred_class[i]==3) & (df_final_age_w_gender.age[i] == 'teens')):
        korrekt +=1
    elif ((df_final_age_w_gender.pred_class[i]==1) & (df_final_age_w_gender.age[i] == 'twenties' or df_final_age_w_gender.age[i] == 'thirties')):
        korrekt +=1
    elif ((df_final_age_w_gender.pred_class[i]==0) & (df_final_age_w_gender.age[i] == 'fourties' or df_final_age_w_gender.age[i] == 'fifties')):
        korrekt +=1
    elif ((df_final_age_w_gender.pred_class[i]==2) & (df_final_age_w_gender.age[i] == 'sixties' or df_final_age_w_gender.age[i] == 'seventies')):
        korrekt +=1
    else:
        falsch +=1
        
print ('Korrekt: ' + str(korrekt) + '/' + str(anzahl_samples))
print ('Falsch: ' + str(falsch)+ '/' + str(anzahl_samples))

# Modell Language Detection

In [17]:
model_lang = keras.models.load_model ('/home/ec2-user/SageMaker/voice_gender_age/Language detection/model.h5', custom_objects = None, compile = True)
print ("Modell eingelesen. Übersicht:")
print ("-----------------------------")
model_lang.summary()
print ("-----------------------------")

# df_test_lang = pd.read_csv ('/home/ec2-user/SageMaker/voice_gender_age/validation_test/test_final_FeaturesExtracted.csv', delimiter = ';')
df_test_lang = pd.read_csv ('/home/ec2-user/SageMaker/voice_gender_age/validation_test/FeaturesExtracted_TestdatenBDC_Language.csv', delimiter = ';')

#print ("Datei eingelesen. Vorschau:")
#print ("-----------------------------")
#print (df_test_lang.head())
#print ("-----------------------------")

list_cn = df_test_lang.columns.values
list_cn = np.delete(list_cn,0)
list_cn = np.delete(list_cn,0)
list_cn = np.delete(list_cn,0)
list_cn = np.delete(list_cn,0)
list_cn = np.delete(list_cn,0)
list_cn = np.delete(list_cn,0)
list_cn

nrows = len(df_test_lang)

for i in range (0,nrows):
    for j in list_cn:
        list_c = df_test_lang[j].iloc[i]
        list_c = list_c.replace('[','')
        list_c = list_c.replace(']','')
        list1 = np.fromstring (list_c, sep=' ',dtype=float)
        df_test_lang[j].iat[i] = list1
        df_test_lang[j].iat[i] = df_test_lang[j].iloc[i].mean()
    if (i % 10000 == 0):
        print ("Reihe "+ str(i) + " fertig")
print ("Konvertierung abgeschlossen.")

print ("-----------------------------")
df_pred_lang = df_test_lang.copy()
del df_pred_lang['Unnamed: 0']
del df_pred_lang['index']
del df_pred_lang['path']
del df_pred_lang['age']
del df_pred_lang['gender']
del df_pred_lang['language']
# del df_pred_lang['file_wave']

df_final_lang = df_test_lang.copy()
df_final_lang['pred_class']=0
df_final_lang['pred_proba']=0

anz_datensaetze = len(df_final_lang)
for i in range (anz_datensaetze):
    prediction = model_lang.predict(df_pred_lang.iloc[[i]])
    prediction_class = model_lang.predict_classes(df_pred_lang.iloc[[i]])
    prediction_proba = model_lang.predict_proba(df_pred_lang.iloc[[i]])
    #print(prediction)
    #print (prediction_class)
    df_final_lang.at[i,'pred_class'] = prediction_class
    #df_final.at[i,'pred_proba'] = prediction_proba
    #print (prediction_proba)
   # print ("----------")
    print('--------------------------------------')
    print('Datensatz Nummer (' + str(i) + '):')
    print('--------------------------------------')
    print('Klasse 0 (DE): ' + str(prediction[0,0]))
    print('Klasse 1 (ES): ' + str(prediction[0,1]))
    print('Klasse 2 (FR): ' + str(prediction[0,2]))
    print('-------------------------------------------')
    print('Prediction als Klasse: ' + str(prediction_class[0]))

print ("-----------------------------")
print (df_final_lang[['language','pred_class']])
print ("-----------------------------")

anzahl_samples = df_final_lang.shape[0]
korrekt = 0
falsch = 0

for i in range (anzahl_samples):
    if ((df_final_lang.pred_class[i]==0) & (df_final_lang.language[i] == 'de')):
        korrekt +=1
    elif ((df_final_lang.pred_class[i]==1) & (df_final_lang.language[i] == 'es')):
        korrekt +=1
    elif ((df_final_lang.pred_class[i]==2) & (df_final_lang.language[i] == 'fr')):
        korrekt +=1
    else:
        falsch +=1
        
print ('Korrekt: ' + str(korrekt) + '/' + str(anzahl_samples))
print ('Falsch: ' + str(falsch)+ '/' + str(anzahl_samples))

Modell eingelesen. Übersicht:
-----------------------------
Model: "sequential_229"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_913 (Dense)            (None, 34)                1190      
_________________________________________________________________
dense_914 (Dense)            (None, 100)               3500      
_________________________________________________________________
dense_915 (Dense)            (None, 100)               10100     
_________________________________________________________________
dense_916 (Dense)            (None, 3)                 303       
Total params: 15,093
Trainable params: 15,093
Non-trainable params: 0
_________________________________________________________________
-----------------------------
Reihe 0 fertig
Konvertierung abgeschlossen.
-----------------------------
--------------------------------------
Datensatz Nummer (0):
---------------------